In [ ]:
from crewai import Agent, Task, Crew, LLM
from crewai_tools import tool
from langchain_community.tools import DuckDuckGoSearchRun
import os
from IPython.display import Markdown

In [ ]:
os.environ["AWS_ACCESS_KEY_ID"] = os.environ.get("AWS_ACCESS_KEY_ID")
os.environ["AWS_SECRET_ACCESS_KEY"] = os.environ.get("AWS_SECRET_ACCESS_KEY")
os.environ["AWS_REGION_NAME"] = "us-east-1"

In [ ]:
@tool('DuckDuckGoSearch')
def search(search_query: str):
    """Search the web for information on a given topic"""
    return DuckDuckGoSearchRun().run(search_query)

In [ ]:
modelId="bedrock/anthropic.claude-3-haiku-20240307-v1:0"

In [ ]:
# Configure the LLM
llm = LLM(model=modelId)

In [ ]:
# Define the Agent
legal_agent = Agent(
    role='Legal Assistant',
    goal='Assistant to provide assistance with legal research',
    backstory="You are an experienced Legal assistant specializing in legal matters of India.",
    verbose=False,
    allow_delegation=False,
    llm=llm,
    tools=[search]  # Tool for online searching
)

In [ ]:
# Define the Task
task = Task(
    description="Based on the user's questions: {questions}, research and advise on the Legal Matters.",
    expected_output="A list of legal advise and answers with brief descriptions.",
    agent=legal_agent
)

In [ ]:
# Create the Crew
crew = Crew(
    agents=[legal_agent],
    tasks=[task],
    verbose=False,
)

In [ ]:
# User input for travel preferences
user_input = {
    "questions": "I want a know about the Indian Special Marraige Act."
}

# Execute the Crew
result = crew.kickoff(inputs=user_input)

In [ ]:
Markdown(result.raw)

In [ ]:
# Enabling Memory in the Agent
crew_with_memory = Crew(
    agents=[legal_agent],
    tasks=[task],
    verbose=True,
    memory=True,  # Enable memory
    embedder={
        "provider": "aws_bedrock",
        "config": {
            "model": "amazon.titan-embed-text-v1",  # Embedding model for memory
            "vector_dimension": 1024
        }
    },
    
)

In [ ]:
# Executing the Crew with Memory
result_with_memory = crew_with_memory.kickoff(inputs=user_input)

In [ ]:
from IPython.display import Markdown

In [ ]:
Markdown(result_with_memory.raw)